# ISE 3230 Final Project

## Goal:

Minimize the total operational cost of meeting the Minerva Park's (located in Columbus, Ohio) electricity demand by determining the optimal on/off status of the biomass plants, the amount of power to be produced by each plant at each month, while adhering to the technical constraints (capacity, fuel, ramp-up/down, and startup costs).

## Data Collection: 

We went to a state level government website to find which power plants in Ohio use biomass in order to generate electricity. We were able to find metrics such as power capacity of the plant at any given time (Pi,m) (in MW) and were able to calculate the maximum amount of electricity (in MWh) for a plant running at full capacity. Lastly, we used data/metrics given about the generators at each plant to determine the ramp up/ramp down costs for each plant. Also, we weren't able to find specific labor and maintenance costs for each plant, so we researched the average labor and maintenance costs in a year. And in order to determine the monthly demand for electricity (in MWh), we used the estimated number of households in Minerva Park multiplied by the average number of KWhs used per household on a monthly basis in order to determine the entire city’s monthly electrical demand.


## Problem Set-Up:

The problem should represent a unit commitment MILP such that we can use binary and decision variables in order to determine when certain facilities should be operational at a given month m such that we can determine the most cost-effective way to power the city of columbus solely off of electricity generated using biomass.

Assuming 1 ton of biomassfuel cost = $229.48


### Plant 1: Sauder Power Plant


- Plant Capacity: 7.2 MWh (892 MW in a month)
- Fuel Consumption Rate: 60 tons per day / 1,748 tons per month
- Fuel Consumption cost: \$401,277 per month
- Ramp up/Ramp down cost: $306 
- Labor and Maintenance Cost: \$2,000,000 (dollars/in one year) 166,667 per month


### Plant 2: Napoleon Biogas

- Plant Capacity: 11.2 MW in da day (347.2 MWh in a month)
- Fuel Consumption Rate: 60 tons per day / 1,748 tons per month
- Fuel Consumption cost: \$401,277 per month
- Ramp up/Ramp down cost: $250
- Labor and Maintenance Cost: 2,000,000 (dollars/in one year) 166,667 per month

### Plant 3: Pixelle Speciality Solutions LLC - Chillicothe Facility

- Plant Capacity: 27.2 MW in a day (843.2 MW in a month)
- Fuel Consumption Rate: 60 tons per day / 1,748 tons per month
- Fuel Consumption cost: \$401,277 per month
- Ramp up/Ramp down cost: 187$
- Labor and Maintenance Cost: 2,000,000 (dollars/in one year) 166,667 per month

### City Electricity Demand (Columbus, Ohio):

- Average household usage (in KWh): 745
- Estimated Number of Households: 803.9
- $745 * 803.9 = 598,905$ MW per month for all Minerva Park households or 598.682 MW per month

### Descision Varaibles

- Let $P_{i,m}$ represent the power produced by plant $i$ in month $m$ in (MW).
- Let $b_{i,m}$ represent a binary variable indicating whether plant $i$ is operational (1) or not (0) in month $m$.
- Let $r_{i}$ represent the cost associated cost of ramping up/ramping down the generator at plant $i$.
- Let $f_{i}$ represent the cost of fuel consumed by plant $i$.
- Let $C_{i}$ represent the maximum plant capacity for plant $i$.
- Let $z_{i,m}$ represent an auxillary variable used to measure whether the binary operational status of plant i changes from month m - 1 to month m. 
- Average Monthly MWh for entire city of Columbus: $294,205$ MW
- Average Labor/Maintenance Cost per month:  $166,667$
- Where plant $i = {1, 2 ,3}$ and month $m = {1,2…,12}$:
$$\min Z = \sum_{m=1}^{12}\sum_{i=1}^{3}(f_{i}b_{i,m} + LM_{i}b_{i,m} + r_{i}z_{i,m})$$

### Constraints

### General Constraints:
- **Plant Capacity Constraint:** 
  $$P_{i,m} \leq C_{i} b_{i,m}$$
  
- **Ramp Up/Ramp Down Constraint:**
  $$|P_{i,m} - P_{i,m-1}| \leq C_{i} z_{i,m}$$
  
- **Non-Negativity Constraint:**
  $$P_{i,m} \geq 0$$
  
- **Operational Status:** 
  $$b_{i,m} \in \{0, 1\}$$
  
### Plant 1 Constraints:
$$P_{1,m} \leq 892 \cdot b_{1,m}$$
$$z_{1,m} \geq b_{1,m} - b_{1,m - 1}$$
$$z_{1,m} \geq b_{1,m - 1} - b_{1,m}$$
$$z_{1,1} \geq b_{1,1}$$
$$P_{1,m} \geq 0$$
$$z_{1,m} \geq 0$$

### Plant 2 Constraints:
$$P_{2,m} \leq 347.2 \cdot b_{2,m}$$
$$z_{2,m} \geq b_{2,m} - b_{2,m - 1}$$
$$z_{2,m} \geq b_{2,m - 1} - b_{2,m}$$
$$z_{2,1} \geq b_{2,1}$$
$$P_{2,m} \geq 0$$
$$z_{2,m} \geq 0$$

### Plant 3 Constraints:
$$P_{3,m} \leq 843.2 \cdot b_{3,m}$$
$$z_{3,m} \geq b_{3,m} - b_{3,m - 1}$$
$$z_{3,m} \geq b_{3,m - 1} - b_{3,m}$$
$$z_{3,1} \geq b_{3,1}$$
$$P_{3,m} \geq 0$$
$$z_{3,m} \geq 0$$

### Demand Constraints:
- The total power production must meet or exceed the demand for each month:
  $$P_{1,m} + P_{2,m} + P_{3,m} \geq \text{demand}_{m}$$

- The total power production must not exceed the maximum demand for each month:
  $$P_{1,m} + P_{2,m} + P_{3,m} \leq 2082$$

### Additional Constraints:
- **At Least Two Plants Must Be Operational:**
  $$\sum_{i=1}^{3} b_{i,m} \geq 2 \quad \forall m$$
  
### Binary Constraints:
$$b_{i,m} \in \{0, 1\} \quad \forall i, m$$



## Minimization Problem

In [63]:
import cvxpy as cp
import numpy as np

# Parameters (constant values for the optimization)
average_demand = 598.90  # MW

# Seasonal multipliers to adjust demand
seasonal_multipliers = np.array([
    1.1,  # January (Winter)
    1.2,  # February (Winter)
    0.9,  # March (Spring)
    0.8,  # April (Spring)
    0.9,  # May (Spring)
    1.2,  # June (Summer)
    1.3,  # July (Summer)
    1.2,  # August (Summer)
    1.0,  # September (Fall)
    0.9,  # October (Fall)
    0.8,  # November (Fall)
    1.1   # December (Winter)
])

# Adjust the demand array based on seasonal variation
demand = average_demand * seasonal_multipliers

# Other parameters remain unchanged
C1, C2, C3 = 892, 347.2, 843.2  # Maximum capacities for each plant
fuel_cost_1, fuel_cost_2, fuel_cost_3 = 401277, 401277, 401277  # Fuel costs per month
ramp_cost_1, ramp_cost_2, ramp_cost_3 = 2448, 2200, 2300  # Ramp-up/down costs per day
LM_cost = 166679  # Labor/Maintenance cost per month per plant

# Maximum demand constraint (2082 MW)
max_demand = 2082

# Decision variables (these will be optimized)
P = cp.Variable((3, 12), integer=True)  # Power produced by each plant for each month
B = cp.Variable((3, 12), boolean=True)  # Operational status of each plant
Z = cp.Variable((3, 12), boolean=True)  # Ramp-up/ramp-down status

# Objective function components
fuel_cost = (
    fuel_cost_1 * cp.sum(P[0, :]) + 
    fuel_cost_2 * cp.sum(P[1, :]) + 
    fuel_cost_3 * cp.sum(P[2, :])
)

ramp_cost = (
    ramp_cost_1 * cp.sum(Z[0, :]) + 
    ramp_cost_2 * cp.sum(Z[1, :]) + 
    ramp_cost_3 * cp.sum(Z[2, :])
)

labor_maintenance_cost = (
    LM_cost * cp.sum(B[0, :]) + 
    LM_cost * cp.sum(B[1, :]) + 
    LM_cost * cp.sum(B[2, :])
)

# Total objective function (Minimize)
total_cost = fuel_cost + ramp_cost + labor_maintenance_cost
objective = cp.Minimize(total_cost)

# Constraints List
constraints = []

# 1. Power Produced by Each Plant Cannot Exceed Maximum Capacity
constraints += [P[0, :] <= C1 * B[0, :]]  # Plant 1 capacity constraint
constraints += [P[1, :] <= C2 * B[1, :]]  # Plant 2 capacity constraint
constraints += [P[2, :] <= C3 * B[2, :]]  # Plant 3 capacity constraint

# 2. Demand Satisfaction: The total power produced by all plants must meet or exceed the demand
constraints += [cp.sum(P, axis=0) >= demand]  # Total power >= demand for each month

# 3. Ramp-up and Ramp-down Constraints
for i in range(3):  # For each plant
    for m in range(1, 12):  # Start from the second month
        constraints += [P[i, m] - P[i, m-1] <= C1 * Z[i, m]]  # Ramp-up constraint
        constraints += [P[i, m-1] - P[i, m] <= C1 * Z[i, m]]  # Ramp-down constraint

# 4. Operational Status Constraints: If a plant is not operational, its power should be zero.
for i in range(3):  # For each plant
    for m in range(12):  # For each month
        constraints += [P[i, m] <= C1 * B[i, m]]  # If plant is off, power should be 0
        constraints += [P[i, m] >= 0]  # Power must be non-negative

# 5. Ensure Binary Nature of B and Z (Binary constraints for operational status and ramp status)
# These are implicitly enforced by the variable declarations (no need for additional constraints)

# 6. New Constraint: At least two plants must be operational each month
for m in range(12):  # For each month
    constraints += [cp.sum(B[:, m]) >= 2]  # Sum of operational plants >= 2

# 7. New Demand Constraint: The total power production should not exceed the maximum demand
constraints += [cp.sum(P, axis=0) <= max_demand]  # Total power should not exceed max demand for each month

# Solve the problem
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.GUROBI, verbose=True, qcp=True)

# After solving, you can access the optimized values for the variables
print("Optimized Power:", P.value)
print("Operational Status:", B.value)
print("Ramp Status:", Z.value)


                                     CVXPY                                     
                                     v1.5.3                                    
(CVXPY) Nov 24 08:51:37 PM: Your problem has 108 variables, 210 constraints, and 0 parameters.
(CVXPY) Nov 24 08:51:37 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 24 08:51:37 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 24 08:51:37 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Nov 24 08:51:37 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Nov 24 08:51:37 PM: Compiling problem (target solver=GUROBI).